<a href="https://colab.research.google.com/github/g0m3zc/Hotel_booking_demand/blob/main/Hotel_Booking_Demand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand/code

Original Source: https://doi.org/10.1016/j.dib.2018.11.126


Reviewing dataset of Hotel Booking for data exploration and preparation for analysis.

In [ ]:
# To fetch csv
import requests
import pandas as pd
from io import StringIO

In [ ]:
orig_url = "https://drive.google.com/file/d/1e766jh27Gu6z8Bqloe9aXxeaUH4QhDiE/view?usp=share_link"
file_id = orig_url.split('/')[-2]
dwn_url = "https://drive.google.com/uc?export=download&id=" + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)

In [ ]:
# Creating a pandas DataFrame using the url
df = pd.read_csv(csv_raw, delimiter=',')
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

**Observations:**
- There are nulls values in country, agent, and company.
- String columns are in an object datatype.  Would like to convert these to a string dtype for potential text analysis.  
- Numbers are in both integer and float dtypes.  
- Some datatypes may need to be converted such as updating month datatype from object to number.  
- May need to consider binning some numerical data and reviewing the number of categories in the object datatypes.
- Can consider creating columns to calculate time between dates.

Exploration of columns.

In [ ]:
# Creating lists with the column names in order to simplify the review of the dataframe by column type.

object_list = []
int64_list = []
float_list = []

for i in df:
  if df[i].dtypes == "object":
    object_list.append(i)
  elif df[i].dtypes == "int64":
    int64_list.append(i)
  else:
    float_list.append(i)

print(object_list, "\n", int64_list, "\n", float_list)

['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 'reservation_status', 'reservation_status_date'] 
 ['is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'babies', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list', 'required_car_parking_spaces', 'total_of_special_requests'] 
 ['children', 'agent', 'company', 'adr']


In [ ]:
# Reviewing object columns to understand how many catagories are in each column.

for i in object_list:
  print(i, df[i].nunique())

hotel 2
arrival_date_month 12
meal 5
country 177
market_segment 8
distribution_channel 5
reserved_room_type 10
assigned_room_type 12
deposit_type 3
customer_type 4
reservation_status 3
reservation_status_date 926


In [ ]:
# Reviewing values in the columns with less than 20 categories.
for i in object_list:
  if df[i].nunique() < 20:
    print(i, df[i].unique(), "\n")

hotel ['Resort Hotel' 'City Hotel'] 

arrival_date_month ['July' 'August' 'September' 'October' 'November' 'December' 'January'
 'February' 'March' 'April' 'May' 'June'] 

meal ['BB' 'FB' 'HB' 'SC' 'Undefined'] 

market_segment ['Direct' 'Corporate' 'Online TA' 'Offline TA/TO' 'Complementary' 'Groups'
 'Undefined' 'Aviation'] 

distribution_channel ['Direct' 'Corporate' 'TA/TO' 'Undefined' 'GDS'] 

reserved_room_type ['C' 'A' 'D' 'E' 'G' 'F' 'H' 'L' 'P' 'B'] 

assigned_room_type ['C' 'A' 'D' 'E' 'G' 'F' 'I' 'B' 'H' 'P' 'L' 'K'] 

deposit_type ['No Deposit' 'Refundable' 'Non Refund'] 

customer_type ['Transient' 'Contract' 'Transient-Party' 'Group'] 

reservation_status ['Check-Out' 'Canceled' 'No-Show'] 



**Observations:**
- Arrival date month can be updated to a numeric value to match other date columns.
- Meal column includes an "undefined" value which may need to be treated as a null.
- All columns appear usable and will be included in dataset.

In [ ]:
# Reviewing the countries in the dataset.
print(df["country"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%')

PRT    40.9%
GBR    10.2%
FRA     8.8%
ESP     7.2%
DEU     6.1%
       ...  
DJI     0.0%
BWA     0.0%
HND     0.0%
VGB     0.0%
NAM     0.0%
Name: country, Length: 177, dtype: object


In [ ]:
# Reviewing the reservation status dates

print(df["reservation_status_date"].value_counts())

2015-10-21    1461
2015-07-06     805
2016-11-25     790
2015-01-01     763
2016-01-18     625
              ... 
2015-02-27       1
2015-04-25       1
2015-03-11       1
2015-06-14       1
2015-02-12       1
Name: reservation_status_date, Length: 926, dtype: int64
